In [54]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import layers
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
import nibabel as nib
from scipy import ndimage
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
np.random.seed(1)

In [55]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

<h2> Tratamiento de los datos </h2>

<h3>Carga de los datos y de las rutas de las imagenes</h3>

En primer lugar elaboraremos un dataframe elaborado con Pandas en el que incluiremos las rutas de las imagenes, su identificación, la clase de cancer, su edad, su tipo de operación si es que ha realizado alguna y su supervivencia.
Tambien serán eliminados los campos que no sean importantes como las identificaciones de otros años.


In [56]:
directory_path = 'G:\Dataset'
os.chdir(directory_path)

survival_path = os.path.join(directory_path, 'survival_info.csv')
survival_dataframe = pd.read_csv(survival_path, header=0, index_col='Brats20ID')
survival_dataframe.index.names = ['ID']

grade_dataframe = os.path.join(directory_path, 'name_mapping.csv')
grade_dataframe = pd.read_csv(grade_dataframe, header=0, index_col='BraTS_2020_subject_ID')
grade_dataframe.index.names = ['ID']

In [57]:
dataframe = grade_dataframe.join(survival_dataframe)
dataframe.drop(columns=['BraTS_2017_subject_ID', 'BraTS_2018_subject_ID', 'TCGA_TCIA_subject_ID', 'BraTS_2019_subject_ID'], inplace=True)
dataframe.rename(columns={'Grade':'grade', 'Age':'age','Survival_days':'survival_days','Extent_of_Resection':'extent_of_resection'}, inplace=True)
dataframe.describe(include = 'all')

,grade,age,survival_days,extent_of_resection
count,369,236.000000,236,129
unique,2,NaN,218,2
top,HGG,NaN,448,GTR
freq,293,NaN,3,119
mean,NaN,61.223203,NaN,NaN
std,NaN,11.874114,NaN,NaN
min,NaN,18.975000,NaN,NaN
25%,NaN,54.244250,NaN,NaN
50%,NaN,61.471000,NaN,NaN
75%,NaN,69.200000,NaN,NaN


In [58]:
def load_image_path(dataframe, directory_path):
    '''
    '''
    images=['_t1.nii', '_t1ce.nii', '_t2.nii', '_flair.nii']
    for ID in dataframe.index:
        dataframe.at[ID, 't1'] = os.path.join(directory_path, ID, ID+images[0])
        dataframe.at[ID, 't1ce'] = os.path.join(directory_path, ID, ID+images[1])
        dataframe.at[ID, 't2'] = os.path.join(directory_path, ID, ID+images[2])
        dataframe.at[ID, 'flair'] = os.path.join(directory_path, ID, ID+images[3])
    return dataframe

In [59]:
dataframe = load_image_path(dataframe, directory_path)
dataframe.head()

,grade,age,survival_days,extent_of_resection,t1,t1ce,t2,flair
ID,,,,,,,,
BraTS20_Training_001,HGG,60.463,289,GTR,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...
BraTS20_Training_002,HGG,52.263,616,GTR,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...
BraTS20_Training_003,HGG,54.301,464,GTR,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...
BraTS20_Training_004,HGG,39.068,788,GTR,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...
BraTS20_Training_005,HGG,68.493,465,GTR,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...


<h3>Preprocesado de los datos</h3>

In [60]:
#Analizando los datos podemos observar que todos los pacientes con un grado de tumor bajo no tienen datos de supervivencia ni de
#extracción del tumor, ni edad. Esto es en gran medida porque estos pacientes tienen un pronostico bastante favorable [1] y la mayoria no necesita cirugia.
#Por eso asumiremos que ninguno de ellos se realizo cirugía, su edad será fijada de forma aleatoria entre el minimo y el maximo de las edades y el tiempo de supervivencia sera aleatorio entre 
#https://www.analesdepediatria.org/es-gliomas-bajo-grado-revision-10-articulo-S1695403314000873#:~:text=La%20supervivencia%20global%20fue%20del%2088%2C3%25%2C%20con%20una,fue%20del%2083%2C7%25.
#https://ascopubs.org/doi/full/10.1200/JOP.2016.018622#:~:text=A%20larger%20study%20of%20216,of%20resection%20was%20%3C%2090%25.

#Segun el estudio [2] un 76% de las personas con una extirpación de menos del 90% del tumor sobrevivieron más de 5 años. Por lo que
#como asumimos que estas personas no se realizaron cirugía ya que no tenemos datos, calcularemos el tiempo de supervivencia como un número
#aleatorio entre 4 y 7 teniendo un 24% de ser entre 4-5 y un 76% de ser entre 5-7
#En cuanto a la edad esta será calcula aleatoriamente en el rango de la media del resto de las edades con la desviacion tipica [media-dt, media+dt] 
dataframe.loc[(dataframe['grade'] != 'HGG') & (pd.isna(dataframe['age']) == True)].head(5)

,grade,age,survival_days,extent_of_resection,t1,t1ce,t2,flair
ID,,,,,,,,
BraTS20_Training_260,LGG,NaN,NaN,NaN,G:\Dataset\BraTS20_Training_260\BraTS20_Traini...,G:\Dataset\BraTS20_Training_260\BraTS20_Traini...,G:\Dataset\BraTS20_Training_260\BraTS20_Traini...,G:\Dataset\BraTS20_Training_260\BraTS20_Traini...
BraTS20_Training_261,LGG,NaN,NaN,NaN,G:\Dataset\BraTS20_Training_261\BraTS20_Traini...,G:\Dataset\BraTS20_Training_261\BraTS20_Traini...,G:\Dataset\BraTS20_Training_261\BraTS20_Traini...,G:\Dataset\BraTS20_Training_261\BraTS20_Traini...
BraTS20_Training_262,LGG,NaN,NaN,NaN,G:\Dataset\BraTS20_Training_262\BraTS20_Traini...,G:\Dataset\BraTS20_Training_262\BraTS20_Traini...,G:\Dataset\BraTS20_Training_262\BraTS20_Traini...,G:\Dataset\BraTS20_Training_262\BraTS20_Traini...
BraTS20_Training_263,LGG,NaN,NaN,NaN,G:\Dataset\BraTS20_Training_263\BraTS20_Traini...,G:\Dataset\BraTS20_Training_263\BraTS20_Traini...,G:\Dataset\BraTS20_Training_263\BraTS20_Traini...,G:\Dataset\BraTS20_Training_263\BraTS20_Traini...
BraTS20_Training_264,LGG,NaN,NaN,NaN,G:\Dataset\BraTS20_Training_264\BraTS20_Traini...,G:\Dataset\BraTS20_Training_264\BraTS20_Traini...,G:\Dataset\BraTS20_Training_264\BraTS20_Traini...,G:\Dataset\BraTS20_Training_264\BraTS20_Traini...


In [61]:
def generate_survival(dataframe):
    '''
    '''
    a=4
    b=5
    c=7
    probability_low = 24
    indexes = dataframe.loc[(pd.isna(dataframe['age']) == True)].index
    survival_fill = pd.DataFrame([0 for i in range(0, len(indexes))], columns=['fill'], index=indexes)
    for i in indexes.values:
        low = np.random.randint((a*365), high=(b*365)+1)
        high = np.random.randint((b*365), high=(c*365)+1)
        probability = np.random.randint(1, high=101)
        
        if probability <= probability_low:
            survival_fill.loc[i, 'fill'] = low
        else:
            survival_fill.loc[i, 'fill'] = high
            
    return survival_fill['fill']

def generate_age(dataframe):
    '''
    '''
    indexes = dataframe.loc[(pd.isna(dataframe['age']) == True)].index
    age_fill = pd.DataFrame([0 for i in range(0, len(indexes))], columns=['fill'], index=indexes)
    
    ages = dataframe.loc[pd.isna(dataframe['age']) == False]
    age_mean = np.round(ages.mean(), 3)
    age_std = np.round(ages.std(), 3)
    minimo = age_mean-age_std
    maximo = age_mean+age_std
    
    for i in indexes.values:
        random = np.round(np.random.uniform(minimo, maximo), 3)
        age_fill.loc[i, 'fill'] = random
        
    return age_fill['fill']

def normalize_age(dataframe):
    '''
    '''
    max_value = dataframe['age'].max()
    min_value = dataframe['age'].min()
    dataframe['age'] = (dataframe['age'] - min_value) / (max_value - min_value)
    return dataframe

In [62]:
dataframe['survival_days'].fillna(generate_survival(dataframe), inplace=True)
dataframe['age'].fillna(generate_age(dataframe), inplace=True)
#dataframe = normalize_age(dataframe)

grade_encoding = {'LGG':0, 'HGG':1}
dataframe['grade'] = dataframe['grade'].replace(grade_encoding)

dataframe.drop(dataframe.loc[dataframe['survival_days'] == 'ALIVE (361 days later)'].index, axis=0, inplace=True)
dataframe = dataframe.astype({"grade": int, 'survival_days':int})

survival_encoding = {'< 1 year':0, '1-5 years':1, '+5 years':2}
for i in dataframe.index:
    if dataframe['survival_days'].loc[i] < 365:
        dataframe.loc[[i],['survival_days']] = survival_encoding['< 1 year']
    elif dataframe['survival_days'].loc[i] < 1825:
        dataframe.loc[[i],['survival_days']] = survival_encoding['1-5 years']
    else:
        dataframe.loc[[i],['survival_days']] = survival_encoding['+5 years']
        
dataframe['extent_of_resection'].fillna('NONE', inplace=True)
dataframe = pd.get_dummies(dataframe, columns=['extent_of_resection'], prefix='resection_')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_6480\3122688624.py:29: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  age_mean = np.round(ages.mean(), 3)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6480\3122688624.py:30: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  age_std = np.round(ages.std(), 3)


In [63]:
dataframe.head()

,grade,age,survival_days,t1,t1ce,t2,flair,resection__GTR,resection__NONE,resection__STR
ID,,,,,,,,,,
BraTS20_Training_001,1,60.463,0,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,G:\Dataset\BraTS20_Training_001\BraTS20_Traini...,1,0,0
BraTS20_Training_002,1,52.263,1,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,G:\Dataset\BraTS20_Training_002\BraTS20_Traini...,1,0,0
BraTS20_Training_003,1,54.301,1,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,G:\Dataset\BraTS20_Training_003\BraTS20_Traini...,1,0,0
BraTS20_Training_004,1,39.068,1,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,G:\Dataset\BraTS20_Training_004\BraTS20_Traini...,1,0,0
BraTS20_Training_005,1,68.493,1,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,G:\Dataset\BraTS20_Training_005\BraTS20_Traini...,1,0,0


In [64]:
diff = dataframe.loc[dataframe['grade'] == 1].count()[0] - dataframe.loc[dataframe['grade'] == 0].count()[0]

In [65]:
for i in range(diff):
    dataframe.drop(dataframe.loc[dataframe['grade'] == 1].index[-1], inplace=True)

<h3>Carga y preprocesado de las imagenes</h3>

In [66]:
#HU -1000 (aire) 800 (huesos son 1000, asi los evitamos)
class Data_generator():
    """
    Data generator
    """
    def __init__(self, dataframe, train_size, width, height, depth, slices=10):
        self.dataframe = dataframe
        self.train_size = train_size
        self.slices = slices
        self.width = width
        self.height = height
        self.depth = depth
        
    def generate_split_indexes(self):
        '''
        '''
        np.random.seed(1)
        permutation = np.random.permutation(len(self.dataframe))
        index = int(len(self.dataframe) * self.train_size)
        train_index = permutation[:index]
        test_index = permutation[index:]
        
        index = int(index * self.train_size)
        train_index, valid_index = train_index[:index], train_index[index:]
        
        return train_index, valid_index, test_index
    
    def read_scan(self, filepath):
        """Read and load volume"""
        # Read file
        img = nib.load(filepath)
        # Rotate the image to a fixed axes
        img = nib.as_closest_canonical(img)
        # Get raw data
        img = img.get_fdata()
        return img
    
    def normalize(self, img):
        """Normalize the volume"""
        minimun = -1000
        maximun = 800
        img[img < minimun] = minimun
        img[img > maximun] = maximun
        #img = (img - minimun) / (maximun - minimun)
        img = img.astype("float32")
        return img
    
    def resize_volume(self, img):
        # Get current depth
        current_depth = img.shape[-1]
        current_width = img.shape[0]
        current_height = img.shape[1]
        # Compute depth factor
        depth = current_depth / self.depth
        width = current_width / self.width
        height = current_height / self.height
        depth_factor = 1 / depth
        width_factor = 1 / width
        height_factor = 1 / height
        
        # Resize across z-axis
        img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
        return img        
    
    def preprocess_image(self, path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        # Read scan
        volume = self.read_scan(path)
        # Normalize
        volume = self.normalize(volume)
        #volume = self.resize_volume(volume)
        
        return volume[:,:,-90::30]
    
    def generate_data(self, indexes, is_training, output, batch_size=32):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        # arrays to store our batched data
        t1, t1ce, t2, flair, text, labels = [], [], [], [], [], []
        while True:
            for index in indexes:
                patient = self.dataframe.iloc[index]
                
                t1_path = patient['t1']
                t1ce_path = patient['t1ce']
                t2_path = patient['t2']
                flair_path = patient['flair']
                for i in range(4):

                    if(i == 0):
                        t1ce.append(self.preprocess_image(t1_path))
                    elif(i == 1):
                        t1ce.append(self.preprocess_image(t1ce_path))
                    elif(i == 2):
                        t1ce.append(self.preprocess_image(t2_path))
                    elif(i == 3):
                        t1ce.append(self.preprocess_image(flair_path))
                    
                    if(output == 'grade'):
                        labels.append(to_categorical(patient['grade'], 2))
                        text.append(patient[['age', 'resection__GTR', 'resection__NONE', 'resection__STR']])
                    else:
                        labels.append(to_categorical(patient['survival_days'], 3))
                        text.append(patient[['age', 'resection__GTR', 'resection__NONE', 'resection__STR']])
                    

                    # Lo que hace yield es crear un generador, es decir, cuando se ejecute este trozo de código no se ejecuta
                    # instantaneamente, sino que más adelante cuando internamente el programa llame a next se ejecutará el código
                    # hasta que se encuentre un yield y ahí parará. Con el siguiente next volvera a hacer una iteración y así.
                    if len(t1ce) >= batch_size:
                        yield [np.float32(t1ce), np.float32(text)], np.float32(labels)
                        t1, t1ce, t2, flair, text, labels = [], [], [], [], [], []
                    
            if not is_training:
                break

In [67]:
train_size = 0.80
image_width = 240
image_height = 240
image_depth_o = 150
slices_step = 50
image_depth = (image_depth_o -1)//slices_step +1

data_generator = Data_generator(dataframe, train_size, image_width, image_height, image_depth, slices_step)
train_index, valid_index, test_index = data_generator.generate_split_indexes() 

In [68]:
def crear_rama_grado(modelo, input_text):
    """
    crea la rama que predice el grado del tumor
    """
    rama_grado = layers.Concatenate()([modelo, input_text])

    rama_grado = layers.Dense(256, activation="relu")(rama_grado)
    rama_grado = layers.BatchNormalization()(rama_grado)
    rama_grado = layers.Dropout(0.3)(rama_grado)
    rama_grado = layers.Dense(512, activation="relu")(rama_grado)
    rama_grado = layers.Dense(2)(rama_grado)
    rama_grado = layers.Activation("softmax", name="grade")(rama_grado)

    return rama_grado

def crear_rama_supervivencia(modelo, input_text):
    """
    crea la rama que predice la supervivencia del paciente
    """
    rama_supervivencia = layers.Concatenate()([modelo, input_text])

    rama_supervivencia = layers.Dense(256, activation="relu")(rama_supervivencia)
    rama_supervivencia = layers.BatchNormalization()(rama_supervivencia)
    rama_supervivencia = layers.Dropout(0.4)(rama_supervivencia)
    rama_supervivencia = layers.Dense(512, activation="relu")(rama_supervivencia)
    rama_supervivencia = layers.Dense(3)(rama_supervivencia)
    rama_supervivencia = layers.Activation("softmax", name="survival")(rama_supervivencia)

    return rama_supervivencia

In [69]:
from keras.applications.densenet import DenseNet121

input_text = layers.Input(shape=(4,))

base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(240,240,3))
base = base_model.output
base = layers.GlobalAveragePooling2D()(base)
rama_grado = crear_rama_grado(base, input_text)

rama_supervivencia = crear_rama_supervivencia(base, input_text)

# Model to be trained
grade_model = Model(inputs=[base_model.input, input_text], outputs=rama_grado, name="grade_model")
survival_model = Model(inputs=[base_model.input, input_text], outputs=rama_supervivencia, name="survival_model")
 
# Training only top layers i.e. the layers which we have added in the end
for layer in base_model.layers:
    layer.trainable = False

In [19]:
epochs = 100
grade_model.compile(optimizer=Adam(learning_rate=0.01), 
            loss="binary_crossentropy",
            metrics = ['accuracy', keras.metrics.Recall()]
)

batch_size = 16
valid_batch_size = 16
train_gen = data_generator.generate_data(train_index, is_training=True, output='grade', batch_size=batch_size)
valid_gen = data_generator.generate_data(valid_index, is_training=True, output='grade', batch_size=valid_batch_size)
earylyStopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, min_delta=0.01)
Reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.01,
)

callbacks = [earylyStopping, Reducer]

grade_history = grade_model.fit(train_gen,
                    steps_per_epoch=(len(train_index)*4)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=(len(valid_index)*4)//valid_batch_size,
                    verbose=1)

Epoch 1/100
24/24 [==============================] - 62s 3s/step - loss: 0.6825 - accuracy: 0.6146 - recall: 0.6146 - val_loss: 0.9361 - val_accuracy: 0.5312 - val_recall: 0.5312
Epoch 2/100
24/24 [==============================] - 63s 3s/step - loss: 0.4567 - accuracy: 0.8047 - recall: 0.8047 - val_loss: 0.5020 - val_accuracy: 0.7604 - val_recall: 0.7604
Epoch 3/100
24/24 [==============================] - 61s 3s/step - loss: 0.3912 - accuracy: 0.8229 - recall: 0.8229 - val_loss: 0.5499 - val_accuracy: 0.7083 - val_recall: 0.7083
Epoch 4/100
24/24 [==============================] - 70s 3s/step - loss: 0.3485 - accuracy: 0.8516 - recall: 0.8516 - val_loss: 0.4713 - val_accuracy: 0.7708 - val_recall: 0.7708
Epoch 5/100
24/24 [==============================] - 112s 5s/step - loss: 0.2819 - accuracy: 0.8828 - recall: 0.8828 - val_loss: 0.3983 - val_accuracy: 0.8333 - val_recall: 0.8333
Epoch 6/100
24/24 [==============================] - 65s 3s/step - loss: 0.2369 - accuracy: 0.9401 - rec

24/24 [==============================] - 155s 6s/step - loss: 0.0091 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 0.1463 - val_accuracy: 0.9375 - val_recall: 0.9375
Epoch 47/100
24/24 [==============================] - 155s 6s/step - loss: 0.0075 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 0.1421 - val_accuracy: 0.9479 - val_recall: 0.9479
Epoch 48/100
24/24 [==============================] - 156s 7s/step - loss: 0.0076 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 0.1462 - val_accuracy: 0.9479 - val_recall: 0.9479
Epoch 49/100
24/24 [==============================] - 156s 6s/step - loss: 0.0060 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 0.1490 - val_accuracy: 0.9375 - val_recall: 0.9375
Epoch 50/100
24/24 [==============================] - 153s 6s/step - loss: 0.0058 - accuracy: 1.0000 - recall: 1.0000 - val_loss: 0.1512 - val_accuracy: 0.9479 - val_recall: 0.9479
Epoch 51/100
24/24 [==============================] - 154s 6s/step - loss: 0.0056 - accuracy: 1.0000 - recal

In [70]:
epochs = 100
grade_model.compile(optimizer=Adam(learning_rate=0.001), 
            loss="binary_crossentropy",
            metrics = ['accuracy', keras.metrics.Recall()]
)

batch_size = 16
valid_batch_size = 16
train_gen = data_generator.generate_data(train_index, is_training=True, output='grade', batch_size=batch_size)
valid_gen = data_generator.generate_data(valid_index, is_training=True, output='grade', batch_size=valid_batch_size)
earylyStopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, min_delta=0.01)
Reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.01,
)

callbacks = [earylyStopping, Reducer]

grade_history = grade_model.fit(train_gen,
                    steps_per_epoch=(len(train_index)*4)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=(len(valid_index)*4)//valid_batch_size,
                    verbose=1)

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
#grade_model.save("C:/Users/Usuario/Desktop/tfg/modelos/DenseNet121-gradeFFFF.h5")
#survival_model.save("C:/Users/Usuario/Desktop/tfg/modelos/DenseNet121-survivalFFFF.h5")

In [43]:
epochs = 100
survival_model.compile(optimizer=Adam(learning_rate=0.001), 
            loss="categorical_crossentropy",
            metrics = ['accuracy', keras.metrics.Recall()]
)

batch_size = 16
valid_batch_size = 16
train_gen = data_generator.generate_data(train_index, is_training=True, output='survival', batch_size=batch_size)
valid_gen = data_generator.generate_data(valid_index, is_training=True, output='survival', batch_size=valid_batch_size)
earylyStopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, min_delta=0.01)
Reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.01,
)
callbacks = [earylyStopping]

survival_history = survival_model.fit(train_gen,
                    steps_per_epoch=(len(train_index)*4)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=(len(valid_index)*4)//valid_batch_size,
                    verbose=1)

Epoch 1/100
24/24 [==============================] - 135s 6s/step - loss: 2.4228 - accuracy: 0.3984 - recall_4: 0.3724 - val_loss: 4.0518 - val_accuracy: 0.4479 - val_recall_4: 0.4479
Epoch 2/100
24/24 [==============================] - 156s 6s/step - loss: 1.1392 - accuracy: 0.5104 - recall_4: 0.3333 - val_loss: 0.8366 - val_accuracy: 0.5833 - val_recall_4: 0.5312
Epoch 3/100
24/24 [==============================] - 155s 6s/step - loss: 0.8698 - accuracy: 0.6250 - recall_4: 0.4375 - val_loss: 1.6374 - val_accuracy: 0.5625 - val_recall_4: 0.5625
Epoch 4/100
24/24 [==============================] - 155s 6s/step - loss: 0.7986 - accuracy: 0.6328 - recall_4: 0.5104 - val_loss: 1.0891 - val_accuracy: 0.5833 - val_recall_4: 0.5521
Epoch 5/100
24/24 [==============================] - 154s 6s/step - loss: 0.7048 - accuracy: 0.6667 - recall_4: 0.5729 - val_loss: 1.3014 - val_accuracy: 0.5417 - val_recall_4: 0.5417
Epoch 6/100
24/24 [==============================] - 154s 6s/step - loss: 0.6532

In [ ]:
epochs = 100
survival_model.compile(optimizer=Adam(learning_rate=0.001), 
            loss="categorical_crossentropy",
            metrics = ['accuracy', keras.metrics.Recall()]
)

batch_size = 16
valid_batch_size = 16
train_gen = data_generator.generate_data(train_index, is_training=True, output='survival', batch_size=batch_size)
valid_gen = data_generator.generate_data(valid_index, is_training=True, output='survival', batch_size=valid_batch_size)
earylyStopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, min_delta=0.01)
Reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.01,
)
callbacks = [earylyStopping]

survival_history = survival_model.fit(train_gen,
                    steps_per_epoch=(len(train_index)*4)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen, 
                    validation_steps=(len(valid_index)*4)//valid_batch_size,
                    verbose=1)

Epoch 1/100
24/24 [==============================] - 88s 4s/step - loss: 1.2790 - accuracy: 0.4297 - recall_7: 0.3255 - val_loss: 2.3840 - val_accuracy: 0.4062 - val_recall_7: 0.3854
Epoch 2/100
24/24 [==============================] - 128s 5s/step - loss: 1.0109 - accuracy: 0.5286 - recall_7: 0.3854 - val_loss: 1.2833 - val_accuracy: 0.5000 - val_recall_7: 0.4062
Epoch 3/100
24/24 [==============================] - 129s 5s/step - loss: 0.9738 - accuracy: 0.5547 - recall_7: 0.4115 - val_loss: 1.0691 - val_accuracy: 0.5312 - val_recall_7: 0.3646
Epoch 4/100
24/24 [==============================] - 130s 5s/step - loss: 0.9419 - accuracy: 0.5703 - recall_7: 0.4297 - val_loss: 0.9670 - val_accuracy: 0.5000 - val_recall_7: 0.3438
Epoch 5/100
24/24 [==============================] - 129s 5s/step - loss: 0.9080 - accuracy: 0.5911 - recall_7: 0.4583 - val_loss: 0.9818 - val_accuracy: 0.4896 - val_recall_7: 0.3750
Epoch 6/100
24/24 [==============================] - 129s 5s/step - loss: 0.8713 

# 5. Evaluación

In [ ]:
def plot_history(history):
    fig, axs = plt.subplots(1, 2,figsize=(16,4))
    
    axs[0].plot(history.history['accuracy'])
    axs[0].plot(history.history['val_accuracy'])
    axs[0].set_title('Precisión del modelo')
    axs[0].set(xlabel='Epoch', ylabel='Perdida')
    axs[0].legend(['Entrenamiento', 'Validación'], loc='upper left')
    
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set_title('Perdida del modelo')
    axs[1].set(xlabel='Epoch', ylabel='Perdida')
    axs[1].legend(['Entrenamiento', 'Validación'], loc='upper left')
    
    plt.show()

In [ ]:
def plot_results(true, pred, encoding):
    pred = np.argmax(pred, axis=1)
    cr_survival = classification_report(true, pred, target_names=encoding.keys())
    print(cr_survival)

In [ ]:
def plot_confusion(true, pred, encoding):
    pred = np.argmax(pred, axis=1)
    cf_matrix = confusion_matrix(true, pred)
    ax = sns.heatmap(cf_matrix, annot=True, fmt='', cmap='Blues')

    ax.set_title('Matriz de confusión\n\n');
    ax.set_xlabel('\nValores predecidos')
    ax.set_ylabel('Valores reales ');

    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(encoding.keys(),)
    ax.yaxis.set_ticklabels(encoding.keys(),)

    ## Display the visualization of the Confusion Matrix.
    plt.show()

In [ ]:
plot_history(grade_history)

In [ ]:
plot_history(survival_history)

In [ ]:
test_batch_size = len(test_index)
test_generator = data_generator.generate_data(test_index, is_training=False, output='grade', batch_size=test_batch_size )
grade_pred = grade_model.predict(test_generator, steps=(len(test_index)*4)//test_batch_size)

In [ ]:
test_batch_size = len(test_index)
test_generator = data_generator.generate_data(test_index, is_training=False, output='survival', batch_size=test_batch_size )
survival_pred = survival_model.predict(test_generator, steps=(len(test_index)*4)//test_batch_size)

In [ ]:
grade_true = []
index = dataframe.iloc[test_index]['grade']
for i in index:
    grade_true.append(i)
    grade_true.append(i)
    grade_true.append(i)
    grade_true.append(i)

survival_true = []
index = dataframe.iloc[test_index]['survival_days']
for i in index:
    survival_true.append(i)
    survival_true.append(i)
    survival_true.append(i)
    survival_true.append(i)

In [ ]:
plot_results(grade_true, grade_pred, grade_encoding)

In [ ]:
plot_confusion(grade_true, grade_pred, grade_encoding)

In [ ]:
plot_results(survival_true, survival_pred, survival_encoding)

In [ ]:
plot_confusion(survival_true, survival_pred, survival_encoding)